In [1]:
%load_ext autoreload
%autoreload

## E-model optimisation with BluePyEModel

This notebook will show how to use BluePyEModel to optimize an e-model using the pipeline.

For more detailed used-cases of BluePyOpt, refer the examples present in the examples directory of BluePyOpt at https://github.com/BlueBrain/BluePyOpt.

Author of this script: Tanguy Damart @ Blue Brain Project

In [2]:
import logging

logger = logging.getLogger()
logging.basicConfig(
    level=logging.INFO,
    handlers=[logging.StreamHandler()]
)

In [3]:
import shutil
import os

if os.path.exists('./config'):
    shutil.rmtree('./config')
    
if os.path.exists('./mechanisms'):
    shutil.rmtree('./mechanisms')
    
shutil.copytree('../../tests/config', './config')
shutil.copytree('../../tests/mechanisms', './mechanisms')

'./mechanisms'

In [4]:
!nrnivmodl mechanisms

which: no xcrun in (/gpfs/bbp.cscs.ch/home/damart/venv/bin:/opt/clmgr/sbin:/opt/clmgr/bin:/opt/sgi/sbin:/opt/sgi/bin:/usr/lib64/qt-3.3/bin:/gpfs/bbp.cscs.ch/home/damart//.nix-profile/bin:/nix/var/nix/profiles/default/bin:/usr/lib64/ccache:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/gpfs/bbp.cscs.ch/ssd/apps/bb5/systemtools:/opt/c3/bin:/opt/ddn/scalers/tools:/usr/lpp/mmfs/bin:/opt/ddn/gs/scripts:/opt/ddn/gs/bin:/opt/ibutils/bin:/opt/ddn/ime/bin:/sbin:/bin:/gpfs/bbp.cscs.ch/home/damart/.local/bin:/gpfs/bbp.cscs.ch/home/damart/bin)
/gpfs/bbp.cscs.ch/home/damart/BluePyEModel/examples/optimisation
Mod files: "mechanisms/CaDynamics_DC0.mod" "mechanisms/Ca_HVA2.mod" "mechanisms/Ca_HVA.mod" "mechanisms/Ca_LVAst.mod" "mechanisms/Ih.mod" "mechanisms/KdShu2007.mod" "mechanisms/K_Pst.mod" "mechanisms/K_Tst.mod" "mechanisms/Nap_Et2.mod" "mechanisms/NaTg2.mod" "mechanisms/NaTg.mod" "mechanisms/SK_E2.mod" "mechanisms/SKv3_1.mod" "mechanisms/StochKv2.mod" "mechanisms/StochKv3.mod"

COBJS=''
 ->

## Instantiate the Pipeline:

The pipeline has three ways of accessing the data required for optimisation: either using an PostrgreSQL database ("sql"), using Nexus ("nexus"), or using the old style config dictionary that relies on recipes ("singlecell"). In this example, we will use this last way, that refer to as "singlecell".

In [5]:
db_api = 'singlecell'

The name of the e-model and species have to be specified. The name of the e-model has to match exactly the entry of the recipes.json when using the "singlecell" data access. With "singlecell", specifying the species of the model is required but inconsequential.

In [6]:
emodel = 'cADpyr_L5TPC'
species = 'rat'

To set where the pipeline should save the results of the optimization as well as to know where are the mechanisms (.mod files) and where to store the final models, some setup is needed:

In [7]:
import pathlib

working_dir = pathlib.Path('./')
mechanisms_dir = working_dir / 'mechanisms'
recipes_path = 'config/recipes.json'
final_path = './final.json'

Finally, with all these information, let's instantiate the pipeline:

In [8]:
from bluepyemodel.emodel_pipeline.emodel_pipeline import EModel_pipeline

pipeline = EModel_pipeline(
    emodel=emodel,
    species=species,
    db_api=db_api,
    recipes_path=recipes_path,
)

## Run the optimisation

To run an optimisation, some parameters have to be set: the maximum number of generation for which the optimization algorithm will run ("max_ngen") and the size of the population ("offspring_size"). For the present example, the number of offspring as well as the maximum number of generations are kept voluntarily low, in practice, typical values would be for example offspring_size = 20 and max_ngen = 500.

For the optimizing algorithm there are three choices: "IBEA", "SO-CMA" and "MO-CMA". "MO-CMA" is recommended by default but the best algorithm can change depending on the problem.

In [9]:
optimizer = "MO-CMA"
max_ngen = 3
opt_params = {
    'offspring_size': 3,
    'weight_hv': 0.4,
    'seed': 42
}

During the optimization, a checkpoint file, containing all the information about the optimisation process will be saved. It's path must also be provided:

In [10]:
chkp_path = f"./checkpoints/checkpoint_{opt_params['seed']}.pkl"

Finally, the optimisation can be ran (warning: slow):

In [ ]:
_ = pipeline.optimize(
        max_ngen=max_ngen,
        stochasticity=False,
        opt_params=opt_params,
        optimizer=optimizer,
        checkpoint_path=chkp_path
    )

INFO:bluepyemodel.emodel_pipeline.emodel_pipeline:Path to the checkpoint file is ./checkpoints/checkpoint_42.pkl
INFO:bluepyemodel.optimisation.optimisation:Running optimisation ...
INFO:__main__:Generation 1
INFO:__main__:gen	nevals	avg    	std    	min    	max    
1  	3     	509.278	193.956	271.314	746.405
INFO:__main__:Generation 2
INFO:__main__:2  	3     	365.574	267.213	155.278	742.634
/gpfs/bbp.cscs.ch/home/damart/venv/lib/python3.8/site-packages/bluepyopt/deapext/hype.py:100: RuntimeWarning: divide by zero encountered in true_divide
  alpha.append(numpy.prod((k - j) / (nrP - j) / i))
INFO:__main__:Generation 3


## Save the best models:

This next step will look at the checkpoint file generated during the optimisation and will save the best model in the final.json whose path was indicated above. This is necessary as for the "singlecell" access point, all subsequent operations (plotting, model management, etc) will except to read the e-models from this file.

In [11]:
pipeline.store_best_model(
    stochasticity=False,
    opt_params=opt_params,
    optimizer=optimizer,
    checkpoint_path=chkp_path
)

INFO:bluepyemodel.api.singlecell:final.json does not exist and will be create


## Plot the convergence of the optimisation, the traces and the scores:

To look at the results of our optimisation, the following function can be used. All figures will be saved in the "figures" directory.

This first plotting function plots the minimum and average fitness as a function of the number of evaluation. It is useful to check if an optimisation succeeded. A succesfull optimisation should present a decreasing fitness.

In [12]:
from bluepyemodel.emodel_pipeline import plotting

plotting.optimization(
    chkp_path, 
    figures_dir="./figures"
)

<Figure size 432x288 with 0 Axes>

This next function reads the best models from the final.json and compute their responses and scores for the protocols used for optimization:

In [13]:
pipeline.plot_models(figures_dir="./figures")

INFO:bluepyemodel.evaluation.evaluation:In compute_responses, 1 emodels found to evaluate.


[{'emodel': 'cADpyr_L5TPC',
  'fitness': 124.89217249398135,
  'parameters': {'constant.distribution_decay': -0.05761755956616788,
   'g_pas.all': 3.0446989142962795e-05,
   'e_pas.all': -66.21521061023577,
   'gIhbar_Ih.somadend': 0.0001468327408282559,
   'gNaTgbar_NaTg.axonal': 1.0557666798267504,
   'gNap_Et2bar_Nap_Et2.axonal': 0.019999999990000003,
   'gK_Pstbar_K_Pst.axonal': 0.21342282914315452,
   'gK_Tstbar_K_Tst.axonal': 0.1450168979063192,
   'gSKv3_1bar_SKv3_1.axonal': 0.4822515777534655,
   'gCa_HVAbar_Ca_HVA2.axonal': 0.00046777231124318225,
   'gCa_LVAstbar_Ca_LVAst.axonal': 0.001192849764132396,
   'gSK_E2bar_SK_E2.axonal': 0.07120480729680471,
   'decay_CaDynamics_DC0.axonal': 49.57316374152761,
   'gamma_CaDynamics_DC0.axonal': 0.005000000022499996,
   'gNaTgbar_NaTg.somatic': 0.2335080803358746,
   'gK_Pstbar_K_Pst.somatic': 0.08338597224852684,
   'gK_Tstbar_K_Tst.somatic': 0.03542971205739827,
   'gSKv3_1bar_SKv3_1.somatic': 0.7939238323813753,
   'gCa_HVAbar_Ca_H

<Figure size 432x288 with 0 Axes>

Using this function, it is also possible to compute the response to any protocols. 

Some eCodes are already defined and can be used as is (e.g: 'APWaveform', 'DeHyperpol', 'FirePattern', 'HyperDepol', 'IDrest', 'IV', 'PosCheops', Ramp', 'sAHP', 'SineSpec', 'SubWhiteNoise'):

In [14]:
additional_protocols = {}
additional_protocols['PosCheops'] = {'type': 'StepThresholdProtocol'}

Protocols of custom length and amplitude can also be used. To know which parameters can be changed for the different protocol, have a look the classes defined in bluepyemodel/ecodes/.

For example, to define a custom step that doesnt not depend on the threshold current: 

(Note the change of the "type". The amplitude is specified in nA).

In [15]:
additional_protocols['Step'] = {
    "type": "StepProtocol",
    'stimuli': {
        'amp': 0.4,
        'delay': 50,
        'duration': 20,
        'totduration': 100,
        'holding_current': -0.1
    } 
}

Another example: a ramp of custom length that depends of the threshold and holding currents.

The thresh_perc field is used to indicate an amplitude relative to the threshold current.

In [16]:
additional_protocols['Ramp'] = {
    "type": "StepThresholdProtocol",
    'stimuli': {
        'thresh_perc': 0.8,
        'delay': 100,
        'duration': 500,
        'totduration': 800
    } 
}

Now let's plot the voltage responses to these stimuli (see the directory ./figures):

In [ ]:
pipeline.plot_models(
        figures_dir="./figures",
        additional_protocols=additional_protocols
    )

INFO:bluepyemodel.evaluation.evaluation:In compute_responses, 1 emodels found to evaluate.
